#### Executing this notebook depends on the Delta tables from project 1 being saved on DBFS!
In project 1, make sure to run chapters 1, 2, 3 fully to save the Delta tables.

### Read data and extract relevant features

In [0]:
import pyspark.sql.functions as F
import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

Most of the data cleaning was already done in project 1 and here we load the cleaned data.

In [0]:
# Read the full dblp fact table.
dblp_df = (spark
           .read.load(f'dbfs:/user/dblpv13/dblp_full')
           .filter(F.col('Lang') == 'en')) # Keep only papers in English

The relevant features for recommending articles are FieldOfStudy, Keywords, Title, Abstract. Since the data was already cleaned in the first project, not much needs to be done here.

In [0]:
# Keep just the relevant features: FOS, Keywords, Title, Abstract.
data = dblp_df.select('FOS', 'Keywords', 'Title', 'Abstract')

# Drop all rows where any value is null.
data = data.na.drop()

# Keep only rows where title and abstract are longer than 1 word.
data = (data.filter( (F.size(F.split(data.Title, ' ')) > 1) & 
                     (F.size(F.split(data.Abstract, ' ')) > 1) ))

# Remove all empty (null or '') values from the FOS and Keywords arrays.
data = (data
        .withColumn('FOS', F.expr('filter(FOS, x -> x is not null)'))
        .withColumn('FOS', F.expr('filter(FOS, x -> length(x) > 0)'))
        .withColumn('Keywords', F.expr('filter(Keywords, x -> x is not null)'))
        .withColumn('Keywords', F.expr('filter(Keywords, x -> length(x) > 0)')))

logger.info(f'Before preprocessing, our data has {data.count()} samples.')

display(data.limit(10))

INFO:py4j.java_gateway:Received command  on object id 
INFO:py4j.java_gateway:Closing down callback connection
INFO:__main__:Before preprocessing, our data has 1356788 samples.


FOS,Keywords,Title,Abstract
"List(Logic synthesis, Topology, Digital electronics, Boolean circuit, Sequential logic, Pass transistor logic, Logic optimization, Algorithm, Register-transfer level, High-level verification, Mathematics)","List(combinational circuits, formal verification, logic design, logic testing, network topology, SAT, combinational circuits, fixed circuit topology, formal verification, logic synthesis, logic verification, minimum ECO, net lists, test vector generation method)",Logic synthesis and verification on fixed topology,"We discuss about logic synthesis and formal verification of combinational circuits mapped to a given fixed topology. Here “fixed topology” means that circuit structures in terms of net lists except for gate/cell types are fixed in advance. That is, for logic synthesis, what should be generated are the types of gates/cells (or simply gates) in the circuits, and all the others are prefixed before synthesis. As the circuit topology is fixed in advance, placement and routing could be shared among different designs and minimum ECO can be realized by keeping the same layout. Also, we can show that we do not need many test vectors in order to guarantee 100% correctness of such synthesis. Small numbers of test vectors, such as only 100 test vectors for 30 input circuits, are sufficient to test if the circuits behave correctly for all possible input value combinations. That is, very efficient formal verification can be realized through simulations with small numbers of test vectors. We present SAT based implementation of the synthesis and a test vector generation method with preliminary but encouraging experimental results."
"List(Detection theory, Computer science, Inference, Algorithm, Artificial intelligence, Fuse (electrical), Machine learning, Recursion)","List(adaptive signal detection, compressed sensing, multidimensional signal processing, signal reconstruction, adaptive sequential compressive signal detection, compressive gain, multidimensional phenomena, nonadaptive strategy, recursive sparse reconstruction algorithm, sequential reconstruction, sparse signal detection, stopping criterion)",Adaptive sequential compressive detection,"Sparsity is at the heart of numerous applications dealing with multidimensional phenomena with low-information content. The primary question that this work investigates is whether, and how much, further compressive gains could be achieved if the goal of the inference task does not require exact reconstruction of the underlying signal. In particular, if the goal is to detect the existence of a sparse signal in noise, it is shown that the number of measurements can be reduced. In contrast to prior work, which considered non-adaptive strategies, a sequential adaptive approach for compressed signal detection is proposed. The key insight is that the decision can be made as soon as a stopping criterion is met during sequential reconstructions. Two sources of performance gains are studied, namely, compressive gains due to adaptation, and computational gains via recursive sparse reconstruction algorithms that fuse newly acquired measurements and previous signal estimates."
"List(Model order selection, Subspace topology, Computer science, Ordinal number, Ordinal data, Matrix decomposition, Speech recognition, Language acquisition, Artificial intelligence, Non-negative matrix factorization, Robot, Machine learning)","List(acoustic signal processing, audio user interfaces, human-robot interaction, intelligent robots, learning (artificial intelligence), matrix decomposition, CSNMF, accuracy improvement, acoustic data augmentation, automatic relevance, command execution, command meaning representation, constrained subspace NMF, grounding information, learning rate improvement, machine learning algorithm, model order selection, ordinal structure, ordinal word acquisition, robot learning, semantic labels, spoken utterances, vocal interface, weakly-supervised NMF, weakly-s

### Preprocessing
We do some baseline preprocessing here, and later perform data analysis to see what else needs to be done.

Load preprocessing libraries.

In [0]:
!pip install spacy

You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command.


In [0]:
!python -m spacy download en_core_web_sm

     |█████████                       |

*** WARNING: max output size exceeded, skipping output. ***

     |████████████████████████████████| 12.8 MB 6.2 MB/s 
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [0]:
import spacy
import re
import string
from pyspark.sql.types import StringType, ArrayType, FloatType
from collections import Counter
import pyspark.sql.functions as F

In [0]:
nlp = spacy.load("en_core_web_sm")

These functions perform the preprocessing of Array or String type columns.

In [0]:
def preprocess(text):
    # remove punctuation
    text = re.sub('[^\w\s]', ' ', text)
    # tokenize text
    text_tokens = text.split(' ')
    # keep only words that are longer than 1 letter
    text_tokens = [word for word in text_tokens if word != '' and len(word) > 1]
    # remove stopwords
    stopwords = nlp.Defaults.stop_words
    tokens = [word for word in text_tokens if not word in stopwords]
    # lowercase and remove custom stopwords
    custom_stopwords = ['doi', 'preprint', 'copyright', 'peer', 'reviewed', 'org', 'https', 'et', 'al', 'author', 'figure','rights', 'reserved', 'permission', 'used', 'using', 'biorxiv', 'medrxiv', 'license', 'fig', 'elsevier', 'pmc', 'czi', 'www', 'paper', '', 'like', 'able']
    tokens = [token.lower() for token in tokens if token not in custom_stopwords]
    return tokens

preprocessUDF = udf(preprocess, ArrayType(StringType()))

def preprocess_list(arr):
    text = ' '.join(arr)
    # remove punctuation
    text = re.sub('[^\w\s]', ' ', text)
    # tokenize text
    text_tokens = text.split(' ')
    # keep only words that are longer than 1 letter
    text_tokens = [word for word in text_tokens if word != '' and len(word) > 1]
    # remove stopwords
    stopwords = nlp.Defaults.stop_words
    tokens = [word for word in text_tokens if not word in stopwords]
    # lowercase and remove custom stopwords
    custom_stopwords = ['doi', 'preprint', 'copyright', 'peer', 'reviewed', 'org', 'https', 'et', 'al', 'author', 'figure','rights', 'reserved', 'permission', 'used', 'using', 'biorxiv', 'medrxiv', 'license', 'fig', 'elsevier', 'pmc', 'czi', 'www', 'paper', '', 'like', 'able']
    tokens = [token.lower() for token in tokens if token not in custom_stopwords]
    return tokens

preprocess_listUDF = udf(preprocess_list, ArrayType(StringType()))

In [0]:
processed_data = data.withColumn("FOS", preprocess_listUDF("FOS")) \
                     .withColumn("Keywords", preprocess_listUDF("Keywords")) \
                     .withColumn("ProcessedTitle", preprocessUDF("Title")) \
                     .withColumn("Abstract", preprocessUDF("Abstract"))

In [0]:
display(processed_data.limit(10))

FOS,Keywords,Title,Abstract,ProcessedTitle
"List(logic, synthesis, topology, digital, electronics, boolean, circuit, sequential, logic, pass, transistor, logic, logic, optimization, algorithm, register, transfer, level, high, level, verification, mathematics)","List(combinational, circuits, formal, verification, logic, design, logic, testing, network, topology, sat, combinational, circuits, fixed, circuit, topology, formal, verification, logic, synthesis, logic, verification, minimum, eco, net, lists, test, vector, generation, method)",Logic synthesis and verification on fixed topology,"List(we, discuss, logic, synthesis, formal, verification, combinational, circuits, mapped, given, fixed, topology, here, fixed, topology, means, circuit, structures, terms, net, lists, gate, cell, types, fixed, advance, that, logic, synthesis, generated, types, gates, cells, simply, gates, circuits, prefixed, synthesis, as, circuit, topology, fixed, advance, placement, routing, shared, different, designs, minimum, eco, realized, keeping, layout, also, need, test, vectors, order, guarantee, 100, correctness, synthesis, small, numbers, test, vectors, 100, test, vectors, 30, input, circuits, sufficient, test, circuits, behave, correctly, possible, input, value, combinations, that, efficient, formal, verification, realized, simulations, small, numbers, test, vectors, we, present, sat, based, implementation, synthesis, test, vector, generation, method, preliminary, encouraging, experimental, results)","List(logic, synthesis, verification, fixed, topology)"
"List(detection, theory, computer, science, inference, algorithm, artificial, intelligence, fuse, electrical, machine, learning, recursion)","List(adaptive, signal, detection, compressed, sensing, multidimensional, signal, processing, signal, reconstruction, adaptive, sequential, compressive, signal, detection, compressive, gain, multidimensional, phenomena, nonadaptive, strategy, recursive, sparse, reconstruction, algorithm, sequential, reconstruction, sparse, signal, detection, stopping, criterion)",Adaptive sequential compressive detection,"List(sparsity, heart, numerous, applications, dealing, multidimensional, phenomena, low, information, content, the, primary, question, work, investigates, compressive, gains, achieved, goal, inference, task, require, exact, reconstruction, underlying, signal, in, particular, goal, detect, existence, sparse, signal, noise, shown, number, measurements, reduced, in, contrast, prior, work, considered, non, adaptive, strategies, sequential, adaptive, approach, compressed, signal, detection, proposed, the, key, insight, decision, soon, stopping, criterion, met, sequential, reconstructions, two, sources, performance, gains, studied, compressive, gains, adaptation, computational, gains, recursive, sparse, reconstruction, algorithms, fuse, newly, acquired, measurements, previous, signal, estimates)","List(adaptive, sequential, compressive, detection)"
"List(model, order, selection, subspace, topology, computer, science, ordinal, number, ordinal, data, matrix, decomposition, speech, recognition, language, acquisition, artificial, intelligence, non, negative, matrix, factorization, robot, machine, learning)","List(acoustic, signal, processing, audio, user, interfaces, human, robot, interaction, intelligent, robots, learning, artificial, intelligence, matrix, decomposition, csnmf, accuracy, improvement, acoustic, data, augmentation, automatic, relevance, command, execution, command, meaning, representation, constrained, subspace, nmf, grounding, information, learning, rate, improvement, machine, learning, algorithm, model, order, selection, ordinal, structure, ordinal, word, acquisition, robot, learning, semantic, labels, spoken, utterances, vocal, interface, weakly, supervised, nmf, weakly, supervised, nonnegative, matrix, factorization, automatic, relevance, determination, ard, language, acquisition, machine, learning, nonnegative, matrix, factorization, nmf, ordinal,

Concatenate the words of each sample into one ArrayType column called 'words'.

In [0]:
processed_data = processed_data.withColumn("words", F.concat(processed_data.FOS, processed_data.Keywords, processed_data.ProcessedTitle, processed_data.Abstract))

In [0]:
display(processed_data.limit(10))

FOS,Keywords,Title,Abstract,ProcessedTitle,words
"List(logic, synthesis, topology, digital, electronics, boolean, circuit, sequential, logic, pass, transistor, logic, logic, optimization, algorithm, register, transfer, level, high, level, verification, mathematics)","List(combinational, circuits, formal, verification, logic, design, logic, testing, network, topology, sat, combinational, circuits, fixed, circuit, topology, formal, verification, logic, synthesis, logic, verification, minimum, eco, net, lists, test, vector, generation, method)",Logic synthesis and verification on fixed topology,"List(we, discuss, logic, synthesis, formal, verification, combinational, circuits, mapped, given, fixed, topology, here, fixed, topology, means, circuit, structures, terms, net, lists, gate, cell, types, fixed, advance, that, logic, synthesis, generated, types, gates, cells, simply, gates, circuits, prefixed, synthesis, as, circuit, topology, fixed, advance, placement, routing, shared, different, designs, minimum, eco, realized, keeping, layout, also, need, test, vectors, order, guarantee, 100, correctness, synthesis, small, numbers, test, vectors, 100, test, vectors, 30, input, circuits, sufficient, test, circuits, behave, correctly, possible, input, value, combinations, that, efficient, formal, verification, realized, simulations, small, numbers, test, vectors, we, present, sat, based, implementation, synthesis, test, vector, generation, method, preliminary, encouraging, experimental, results)","List(logic, synthesis, verification, fixed, topology)","List(logic, synthesis, topology, digital, electronics, boolean, circuit, sequential, logic, pass, transistor, logic, logic, optimization, algorithm, register, transfer, level, high, level, verification, mathematics, combinational, circuits, formal, verification, logic, design, logic, testing, network, topology, sat, combinational, circuits, fixed, circuit, topology, formal, verification, logic, synthesis, logic, verification, minimum, eco, net, lists, test, vector, generation, method, logic, synthesis, verification, fixed, topology, we, discuss, logic, synthesis, formal, verification, combinational, circuits, mapped, given, fixed, topology, here, fixed, topology, means, circuit, structures, terms, net, lists, gate, cell, types, fixed, advance, that, logic, synthesis, generated, types, gates, cells, simply, gates, circuits, prefixed, synthesis, as, circuit, topology, fixed, advance, placement, routing, shared, different, designs, minimum, eco, realized, keeping, layout, also, need, test, vectors, order, guarantee, 100, correctness, synthesis, small, numbers, test, vectors, 100, test, vectors, 30, input, circuits, sufficient, test, circuits, behave, correctly, possible, input, value, combinations, that, efficient, formal, verification, realized, simulations, small, numbers, test, vectors, we, present, sat, based, implementation, synthesis, test, vector, generation, method, preliminary, encouraging, experimental, results)"
"List(detection, theory, computer, science, inference, algorithm, artificial, intelligence, fuse, electrical, machine, learning, recursion)","List(adaptive, signal, detection, compressed, sensing, multidimensional, signal, processing, signal, reconstruction, adaptive, sequential, compressive, signal, detection, compressive, gain, multidimensional, phenomena, nonadaptive, strategy, recursive, sparse, reconstruction, algorithm, sequential, reconstruction, sparse, signal, detection, stopping, criterion)",Adaptive sequential compressive detection,"List(sparsity, heart, numerous, applications, dealing, multidimensional, phenomena, low, information, content, the, primary, question, work, investigates, compressive, gains, achieved, goal, inference, task, require, exact, reconstruction, underlying, signal, in, particular, goal, detect, existence, sparse, signal, noise, shown, number, measurements, reduced, in, contrast, prior, work, considered, non, adaptive, strategies, sequential,

### Data analysis...
... is done on the concatenated column (words of title, abstract, FOS, keywords together).

Let's count the number of words per sample.

In [0]:
processed_data = (processed_data
                  .withColumn('word_count', F.size(F.col('words'))))

Ordered by the count, descendingly.

In [0]:
display(processed_data.orderBy(F.desc('word_count')).limit(10))

words word_count List(prime, order, theory, block, size, discrete, mathematics, pairwise, comparison, combinatorics, incomplete, block, orthogonality, incomplete, block, design, mathematics, recursion, adjusted, orthogonality, existence, result, pairwise, balanced, design, closure, balanced, incomplete, block, design, block, size, recursive, construction, necessary, condition, prime, powers, equivalent, extra, restriction, v, possible, exception, existence, obibds, k, 4, nesting, methods, developed, recursive, construction, sets, balanced, incomplete, block, designs, adjusted, orthogonality, obibds, applied, designs, block, size, 4, 2, sets, treatments, the, methods, extended, include, designs, blocks, size, 4, nested, sub, blocks, size, 2, we, complete, existence, results, greig, rees, obibds, block, size, 3, 2, sets, treatments, showing, necessary, conditions, sufficient, existence, some, pairwise, balanced, design, closures, given, 181, possible, exceptions, k, 8, 9, 29, possible, exceptions, k, set, prime, powers, equivalent, 0, 1, mod, 8, k, set, prime, powers, equivalent, 0, 1, mod, 4, excluding, 4, 5, list, 26, possible, exceptions, extra, restriction, v, 4, mod, 8, keywords, nested, bibd, obibd, onbibd, pbd, 1, introduction, the, designs, consideration, generalisation, balanced, incomplete, block, designs, bibds, single, set, v, treatments, replaced, s, sets, v, forms, bibd, v, k, λ, we, superimpose, s, bibds, block, set, applied, order, 1, bk, array, v, treatments, regarded, replaced, s, bk, ordered, array, single, treatment, plot, regarded, replaced, array, s, treatments, drawn, set, given, order, the, designs, shall, study, paper, constitute, sub, class, orthogonal, balanced, incomplete, block, design, obibd, 24, definition, 1, an, obibd, design, blocks, size, k, s, 1, treatment, factors, size, v, the, blocks, labeled, value, 1, set, treatments, second, set, treatments, forth, labeled, 2, 3, s, 1, in, usual, bibd, setting, blocks, treatments, relationship, represented, incidence, matrix, here, incidence, matrices, required, the, v, b, incidence, matrix, th, set, treatments, respect, blocks, denoted, n, 1, 1, n, 1, j, 1, p, 1, 1, p, j, 1, 1, t, j, 0, j, represents, incidence, matrix, th, set, respect, j, th, set, columns, corresponding, the, v, bk, incidence, matrix, th, set, treatments, respect, plots, denoted, p, 1, 1, the, designs, satisfy, following, properties, 1, each, sets, treatments, arranged, respect, single, blocking, factor, balanced, incomplete, block, design, bibd, identical, parameters, v, k, λ, n, 21, n, 21, t, n, 31, n, 31, t, r, λ, i, λj, replication, r, concurrence, parameter, λ, i, v, v, identity, matrix, j, v, v, matrix, 1, s, 2, the, pairs, treatment, sets, possess, following, property, called, adjusted, orthogonality, 13, 14, orthogonality, short, p, i1, p, j1, t, 1, k, n, i1, n, j1, t, 0, j, 1, we, consider, case, underlying, v, k, λ, bibds, λ, k, 1, r, v, 1, in, case, morgan, 22, lemma, 2, 2, theorem, 2, 5, shown, optimal, statistical, efficiency, designs, occurs, treatment, given, set, appears, exactly, plot, treatment, bar, set, appears, block, exactly, k, times, treatment, bar, set, giving, numbering, sets, assumed, treatment, x, set, appears, treatment, x, set, this, class, designs, study, article, denote, obibd, v, k, λ, s, λ, k, 1, example, 2, an, obibd, 17, 4, 3, 4, given, 4, blocks, developed, modulo, 17, the, blocks, enclosed, parentheses, plots, separated, semi, colons, plot, treatment, sets, given, order, treatment, set, z, 17, note, subsequent, examples, vector, s, treatments, plot, presented, row, vector, column, vector, the, blocks, form, near, resolution, class, treatment, sets, 1, 3, 9, 10, 13, 5, 15, 11, 16, 14, 8, 7, 4, 12, 2, 6, 3, 9, 10, 13, 5, 15, 11, 16, 14, 8, 7, 4, 12, 2, 6, 1, 9, 10, 13, 5, 15, 11, 16, 14, 8, 7, 4, 12, 2, 6, 1, 3, 10, 13, 5, 15, 11, 16, 14, 8, 7, 4, 12, 2, 6, 1, 3, 9, remark, 3, treatment, x, set, occurs, plot, block, treatment, x, set, j, distinction

Let's keep only samples that have up to 400 words.

In [0]:
processed_data = processed_data.filter(F.col('word_count') <= 400)

Let's at the most common words to identify more stopwords.

The first time we ran this, we found words *paper*, *like*, *able* which we added to the stopwords lists above. So, these words won't show up now.

In [0]:
common_words = (processed_data
                .withColumn('expl', F.explode(F.col('words')))
                .groupBy(F.col('expl'))
                .count()
                .orderBy(F.desc('count')))

In [0]:
display(common_words.limit(10))

expl,count
computer,1627900
the,1621401
system,1425698
data,1425501
based,1287227
model,1120256
algorithm,1106717
network,1096161
science,1086033
time,923510


In [0]:
processed_data = processed_data.drop(F.col('word_count'))

In [0]:
logger.info(f'After preprocessing, we have {processed_data.count()} samples.')

INFO:__main__:After preprocessing, we have 1355807 samples.


### Vectorization

In [0]:
from pyspark.ml.feature import HashingTF, IDF

In [0]:
ht = HashingTF(inputCol="words", outputCol="raw_features", numFeatures=1024)
tf_vectors = ht.transform(processed_data.select("title", "words"))
display(tf_vectors.limit(10))

In [0]:
idf = IDF(inputCol="raw_features", outputCol="features", minDocFreq=5)
idf_model = idf.fit(tf_vectors)
tfIdf_vectors = idf_model.transform(tf_vectors)

In [0]:
transformed_data = tfIdf_vectors.select("title", "words", "features").cache()

In [0]:
display(transformed_data.limit(10))

### Clustering

In [0]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.ml.feature import Normalizer, PCA

First, perform PCA.

In [0]:
# using PCA features for clustering
pca = PCA(k=2, inputCol="features", outputCol="pca_features")
pca_model = pca.fit(transformed_data)
transformed_data = pca_model.transform(transformed_data).cache()
display(transformed_data.limit(10))

Then, do the elbow method to determine the best k for K-means clustering.

In [0]:
import gc

cost = []

for k in range(2, 7):
    kmeans = KMeans(k=k, seed=3, featuresCol='pca_features', initMode="random")
    model = kmeans.fit(transformed_data)
    clusteredData = model.transform(transformed_data)
    evaluator = ClusteringEvaluator()
    cost.append((k, evaluator.evaluate(clusteredData)))
    gc.collect()

In [0]:
from matplotlib import pyplot as plt

x, y = zip(*cost)
plt.plot(x, y)
plt.title("Elbow plot")
plt.xlabel("number of clusters")
plt.ylabel("score")
plt.xticks(x)
plt.show()

Looks like 4 is a good choice of k.

Let's fit the model.

In [0]:
kmeans = KMeans(k=4, seed=3, featuresCol='pca_features', initMode='random')
model = kmeans.fit(transformed_data)

In [0]:
clusteredData = model.transform(transformed_data)

In [0]:
display(clusteredData
        .groupby('prediction')
        .count()
        .orderBy(F.asc('prediction'))
        .withColumnRenamed('prediction', 'cluster')
        .limit(10))

### Search Engine

In [0]:
from pyspark.sql.types import DoubleType

def get_similarity_func(query_vector):
    @udf(DoubleType())
    def dot_product(x):
        return float(x.dot(query_vector))
    
    return dot_product


def recommend(title, data, k=10):
    # L2 normalized vectors (for calculating cosine similarity)
    normalizer = Normalizer(p=2, inputCol="features", outputCol="norm_features")
    normalized_data = normalizer.transform(data)
    
    # our target
    # if duplicate titles exist take first
    row = normalized_data.filter(F.col("title") == title).take(1)[0]
    query_vector = row.norm_features
    cluster = row.prediction
    
    # dot product between our target vector and other vectors, resulting in cosine similarity
    similarity_func = get_similarity_func(query_vector)
    
    return (normalized_data
            .filter(F.col("title") != title) # remove entries with same titles
            .filter(F.col("prediction") == cluster) # choose from the same cluster
            .select("title", similarity_func(F.col("norm_features")).alias("similarity"))
            .orderBy(F.desc('similarity'))
            .take(k))

Pass a title and the clustered data to the *recommend* function. It will return a dataframe of articles similar to the title. Returns 10 articles by default.

In [0]:
display(recommend("Logic synthesis and verification on fixed topology", clusteredData))

In [0]:
display(recommend("Audio-based gender and age identification", clusteredData))